# The YUSAG Football Model
    by Matt Robinson, matthew.robinson@yale, Yale Undergraduate Sports Analytics Group
    
This notebook introduces the model we at the Yale Undergraduate Sports Analytics Group (YUSAG) use for our college football rankings. This specific notebook details our FBS rankings at the beginning of the 2017 season.


In [1]:
import numpy as np
import pandas as pd
import math

Let's start by reading in the NCAA FBS football data from 2013-2016:

In [2]:
df_1 = pd.read_csv('NCAA_FBS_Results_2013_.csv')
df_2 = pd.read_csv('NCAA_FBS_Results_2014_.csv')
df_3 = pd.read_csv('NCAA_FBS_Results_2015_.csv')
df_4 = pd.read_csv('NCAA_FBS_Results_2016_.csv')

df = pd.concat([df_1,df_2,df_3,df_4],ignore_index=True)

In [3]:
df.head()

,year,month,day,team,opponent,location,teamscore,oppscore,OT,D1
0,2013,9,7,Air Force,Utah St.,H,20,52,NaN,2
1,2013,9,13,Air Force,Boise St.,V,20,42,NaN,2
2,2013,9,21,Air Force,Wyoming,H,23,56,NaN,2
3,2013,9,28,Air Force,Nevada,V,42,45,NaN,2
4,2013,10,5,Air Force,Navy,V,10,28,NaN,2


As you can see, the `OT` column has some `NaN` values that we will replace with 0.

In [4]:
# fill missing data with 0
df = df.fillna(0)

In [5]:
df.head()

,year,month,day,team,opponent,location,teamscore,oppscore,OT,D1
0,2013,9,7,Air Force,Utah St.,H,20,52,0.0,2
1,2013,9,13,Air Force,Boise St.,V,20,42,0.0,2
2,2013,9,21,Air Force,Wyoming,H,23,56,0.0,2
3,2013,9,28,Air Force,Nevada,V,42,45,0.0,2
4,2013,10,5,Air Force,Navy,V,10,28,0.0,2


I'm also going to make some weights for when we run our linear regression. I have found that using the factorial of the difference between the year and 2012 seems to work decently well. Clearly, the most recent seasons are weighted quite heavily in this scheme.

In [6]:
# update the weights based on a factorial scheme
df['weights'] = (df['year']-2012)
df['weights'] = df['weights'].apply(lambda x: math.factorial(x))

And now, we also are going to make a `scorediff` column that we can use in our linear regression.

In [7]:
df['scorediff'] = (df['teamscore']-df['oppscore'])

In [8]:
df.head()

,year,month,day,team,opponent,location,teamscore,oppscore,OT,D1,weights,scorediff
0,2013,9,7,Air Force,Utah St.,H,20,52,0.0,2,1,-32
1,2013,9,13,Air Force,Boise St.,V,20,42,0.0,2,1,-22
2,2013,9,21,Air Force,Wyoming,H,23,56,0.0,2,1,-33
3,2013,9,28,Air Force,Nevada,V,42,45,0.0,2,1,-3
4,2013,10,5,Air Force,Navy,V,10,28,0.0,2,1,-18


Since we need numerical values for the linear regression algorithm, I am going to replace the locations with what seem like reasonable numbers:
* Visiting = -1
* Neutral = 0
* Home = 1

The reason we picked these exact numbers will become clearer in a little bit.

In [9]:
df['location'] = df['location'].replace('V',-1)
df['location'] = df['location'].replace('N',0)
df['location'] = df['location'].replace('H',1)

In [10]:
df.head()

,year,month,day,team,opponent,location,teamscore,oppscore,OT,D1,weights,scorediff
0,2013,9,7,Air Force,Utah St.,1,20,52,0.0,2,1,-32
1,2013,9,13,Air Force,Boise St.,-1,20,42,0.0,2,1,-22
2,2013,9,21,Air Force,Wyoming,1,23,56,0.0,2,1,-33
3,2013,9,28,Air Force,Nevada,-1,42,45,0.0,2,1,-3
4,2013,10,5,Air Force,Navy,-1,10,28,0.0,2,1,-18


The way our linear regression model works is a little tricky to code up in scikit-learn. It's much easier to do in R, but then you don't have a full understanding of what's happening when we make the model.

In simplest terms, our model predicts the score differential (`scorediff`) of each game based on three things: the strength of the `team`, the strength of the `opponent`, and the `location`.

You'll notice that the `team` and `opponent` features are categorical, and thus are not curretnly ripe for use with linear regression. However, we can use what is called 'one hot encoding' in order to transform these features into a usable form. One hot encoding works by taking the `team` feature, for example, and transforming it into many features such as `team_Yale` and `team_Harvard`. This `team_Yale` feature will usally equal zero, except when the team is actually Yale, then `team_Yale` will equal 1. In this way, it's a binary encoding (which is actually very useful for us as we'll see later).

One can use `sklearn.preprocessing.OneHotEncoder` for this task, but I am going to use Pandas instead: 

In [11]:
# create dummy variables, need to do this in python b/c does not handle automatically like R
team_dummies = pd.get_dummies(df.team, prefix='team')
opponent_dummies = pd.get_dummies(df.opponent, prefix='opponent')

df = pd.concat([df, team_dummies, opponent_dummies], axis=1)

In [12]:
df.head()

,year,month,day,team,opponent,location,teamscore,oppscore,OT,D1,...,opponent_Virginia,opponent_Virginia Tech,opponent_Wake Forest,opponent_Washington,opponent_Washington St.,opponent_West Virginia,opponent_Western Ky.,opponent_Western Mich.,opponent_Wisconsin,opponent_Wyoming
0,2013,9,7,Air Force,Utah St.,1,20,52,0.0,2,...,0,0,0,0,0,0,0,0,0,0
1,2013,9,13,Air Force,Boise St.,-1,20,42,0.0,2,...,0,0,0,0,0,0,0,0,0,0
2,2013,9,21,Air Force,Wyoming,1,23,56,0.0,2,...,0,0,0,0,0,0,0,0,0,1
3,2013,9,28,Air Force,Nevada,-1,42,45,0.0,2,...,0,0,0,0,0,0,0,0,0,0
4,2013,10,5,Air Force,Navy,-1,10,28,0.0,2,...,0,0,0,0,0,0,0,0,0,0


Now let's make our training data, so that we can construct the model. At this point, I am going to use all the avaiable data to train the model, using our predetermined hyperparameters. This way, the model is ready to make predictions for the 2017 season.

In [13]:
# make the training data
X = df.drop(['year','month','day','team','opponent','teamscore','oppscore','D1','OT','weights','scorediff'], axis=1)
y = df['scorediff']
weights = df['weights']

In [14]:
X.head()

,location,team_Air Force,team_Akron,team_Alabama,team_Appalachian St.,team_Arizona,team_Arizona St.,team_Arkansas,team_Arkansas St.,team_Army West Point,...,opponent_Virginia,opponent_Virginia Tech,opponent_Wake Forest,opponent_Washington,opponent_Washington St.,opponent_West Virginia,opponent_Western Ky.,opponent_Western Mich.,opponent_Wisconsin,opponent_Wyoming
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y.head()

0   -32
1   -22
2   -33
3    -3
4   -18
Name: scorediff, dtype: int64

In [16]:
weights.head()

0    1
1    1
2    1
3    1
4    1
Name: weights, dtype: int64

Now let's train the linear regression model. You'll notice that I'm actually using ridge regression (adds an l2 penalty with alpha = 1.0) because that prevents the model from overfitting and also limits the values of the coefficients to be more interpretable. If I did not add this penalty, the coefficients would be huge.

In [17]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge()
ridge_reg.fit(X, y, sample_weight=weights)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [18]:
# get the R^2 value
r_squared = ridge_reg.score(X, y, sample_weight=weights)
print('R^2 on the training data:')
print(r_squared)

R^2 on the training data:
0.495412735743


Now that the model is trained, we can use it to provide our rankings. Note that in this model, a team's ranking is simply defined as its linear regression coefficient, which we call the YUSAG coefficient. 

When predicting a game's score differential on a neutral field, the predicted score differential (`scorediff`) is just the difference in YUSAG coefficients. The reason this works is the binary encoding we did earlier.

#### More details below on how it actually works

Ok, so you may have noticed that every game in our dataframe is actually duplicated, just with the `team` and `opponent` variables switched. This may have seemed like a mistake but it is actually useful for making the model more interpretable. 

When we run the model, we get a coefficient for the `team_Yale` variable, which we call the YUSAG coefficient, and a coefficient for the `opponent_Yale` variable. Since we allow every game to be repeated, these variables end up just being negatives of each other. 

So let's think about what we are doing when we predict the score differential for the Harvard-Penn game with `team` = Harvard and `opponent` = Penn.

In our model, the coefficients are as follows:
- team_Harvard_coef = 7.78
- opponent_Harvard_coef = -7.78
- team_Penn_coef = 6.68
- opponent_Penn_coef = -6.68

when we go to use the model for this game, it looks like this:

`scorediff` = (location_coef $*$ `location`) + (team_Harvard_coef $*$ `team_Harvard`) + (opponent_Harvard_coef $*$ `opponent_Harvard`) + (team_Penn_coef $*$ `team_Penn`) + (opponent_Penn_coef $*$ `opponent_Penn`) + (team_Yale_coef $*$ `team_Yale`) + (opponent_Yale_coef $*$ `opponent_Yale`) + $\cdots$

where the $\cdots$ represent data for many other teams, which will all just equal $0$.

To put numbers in for the variables, the model looks like this:

`scorediff` = (location_coef $*$ $0$) + (team_Harvard_coef $*$ $1$) + (opponent_Harvard_coef $*$ $0$) + (team_Penn_coef $*$ $0$) + (opponent_Penn_coef $*$ $1$) + (team_Yale_coef $*$ $0$) + (opponent_Yale_coef $*$ $0$) + $\cdots$

Which is just:

`scorediff` = (location_coef $*$ $0$) + (7.78 $*$ $1$) + (-6.68 $*$ $1$) = $7.78 - 6.68$ = Harvard_YUSAG_coef - Penn_YUSAG_coef

Thus showing how the difference in YUSAG coefficients is the same as the predicted score differential. Furthermore, the higher YUSAG coefficient a team has, the better they are.

Lastly, if the Harvard-Penn game was to be home at Harvard, we would just add the location_coef:

`scorediff` = (location_coef $*$ $1$) + (team_Harvard_coef $*$ $1$) + (opponent_Penn_coef $*$ $1$) = $1.77 + 7.78 - 6.68$ = Location_coef + Harvard_YUSAG_coef - Penn_YUSAG_coef


In [19]:
# get the coefficients for each feature
coef_data = list(zip(X.columns,ridge_reg.coef_))
coef_df = pd.DataFrame(coef_data,columns=['feature','feature_coef'])
coef_df.head()

,feature,feature_coef
0,location,2.408739
1,team_Air Force,0.077826
2,team_Akron,-13.312718
3,team_Alabama,33.161275
4,team_Appalachian St.,3.490361


Let's get only the team variables, so that it is a proper ranking

In [20]:
# first get rid of opponent_ variables
team_df = coef_df[~coef_df['feature'].str.contains("opponent")]

# get rid of the location variable
team_df = team_df.iloc[1:]

In [21]:
team_df.head()

,feature,feature_coef
1,team_Air Force,0.077826
2,team_Akron,-13.312718
3,team_Alabama,33.161275
4,team_Appalachian St.,3.490361
5,team_Arizona,-3.570632


In [22]:
# rank them by coef, not alphabetical order
ranked_team_df = team_df.sort_values(['feature_coef'],ascending=False)
# reset the indices at 0
ranked_team_df = ranked_team_df.reset_index(drop=True);

In [23]:
ranked_team_df.head()

,feature,feature_coef
0,team_Alabama,33.161275
1,team_Ohio St.,29.246981
2,team_Clemson,29.133253
3,team_Michigan,26.488736
4,team_Washington,24.610815


I'm goint to change the name of the columns and remove the 'team_' part of every string:

In [24]:
ranked_team_df.rename(columns={'feature':'team', 'feature_coef':'YUSAG_coef'}, inplace=True)
ranked_team_df['team'] = ranked_team_df['team'].str.replace('team_', '')

In [25]:
ranked_team_df.head()

,team,YUSAG_coef
0,Alabama,33.161275
1,Ohio St.,29.246981
2,Clemson,29.133253
3,Michigan,26.488736
4,Washington,24.610815


Lastly, I'm just going to shift the index to start at 1, so that it corresponds to the ranking.

In [26]:
ranked_team_df.index = ranked_team_df.index + 1 

In [27]:
ranked_team_df.to_csv("FBS_power_rankings.csv")

## Additional stuff: Testing the model

This section is mostly about how own could test the performance of the model or how one could choose appropriate hyperparamters.

#### Creating a new dataframe

First let's take the original dataframe and sort it by date, so that the order of games in the dataframe matches the order the games were played.

In [28]:
# sort by date and reset the indices to 0
df_dated = df.sort_values(['year', 'month','day'], ascending=[True, True, True])
df_dated = df_dated.reset_index(drop=True)

In [29]:
df_dated.head()

,year,month,day,team,opponent,location,teamscore,oppscore,OT,D1,...,opponent_Virginia,opponent_Virginia Tech,opponent_Wake Forest,opponent_Washington,opponent_Washington St.,opponent_West Virginia,opponent_Western Ky.,opponent_Western Mich.,opponent_Wisconsin,opponent_Wyoming
0,2013,8,29,Akron,UCF,-1,7,38,0.0,2,...,0,0,0,0,0,0,0,0,0,0
1,2013,8,29,Bowling Green,Tulsa,1,34,7,0.0,2,...,0,0,0,0,0,0,0,0,0,0
2,2013,8,29,Fresno St.,Rutgers,1,52,51,1.0,2,...,0,0,0,0,0,0,0,0,0,0
3,2013,8,29,Hawaii,Southern California,1,13,30,0.0,2,...,0,0,0,0,0,0,0,0,0,0
4,2013,8,29,Minnesota,UNLV,1,51,23,0.0,2,...,0,0,0,0,0,0,0,0,0,0


Let's first make a dataframe with training data (the first three years of results)

In [30]:
thirteen_df = df_dated.loc[df_dated['year']==2013]
fourteen_df = df_dated.loc[df_dated['year']==2014]
fifteen_df = df_dated.loc[df_dated['year']==2015]

train_df = pd.concat([thirteen_df,fourteen_df,fifteen_df], ignore_index=True)

Now let's make an initial testing dataframe with the data from this past year.

In [31]:
sixteen_df = df_dated.loc[df_dated['year']==2016]
seventeen_df = df_dated.loc[df_dated['year']==2017]

test_df = pd.concat([sixteen_df,seventeen_df], ignore_index=True)

I am now going to set up a testing/validation scheme for the model. It works like this:

First I start off where my training data is all games from 2012-2015. Using the model trained on this data, I then predict games from the first week of the 2016 season and look at the results.

Next, I add that first week's worth of games to the training data, and now I train on all 2012-2015 results plus the first week from 2016. After training the model on this data, I then test on the second week of games. I then add that week's games to the training data and repeat the same procedure week after week.

In this way, I am never testing on a result that I have trained on. Though, it should be noted that I have also used this as a validation scheme, so I have technically done some sloppy 'data snooping' and this is not a great predictor of my generalization error. 

In [32]:
def train_test_model(train_df, test_df):

    # make the training data
    X_train = train_df.drop(['year','month','day','team','opponent','teamscore','oppscore','D1','OT','weights','scorediff'], axis=1)
    y_train = train_df['scorediff']
    weights_train = train_df['weights']
    
    # train the model
    ridge_reg = Ridge()
    ridge_reg.fit(X_train, y_train, weights_train)
    fit = ridge_reg.score(X_train,y_train,sample_weight=weights_train)
    print('R^2 on the training data:')
    print(fit)
    
    # get the test data
    X_test = test_df.drop(['year','month','day','team','opponent','teamscore','oppscore','D1','OT','weights','scorediff'], axis=1)
    y_test = test_df['scorediff']
    
    # get the metrics
    compare_data = list(zip(ridge_reg.predict(X_test),y_test))
    
    right_count = 0
    for tpl in compare_data:
        if tpl[0] >= 0 and tpl[1] >=0:
            right_count = right_count + 1
        elif tpl[0] <= 0 and tpl[1] <=0:
            right_count = right_count + 1
    accuracy = right_count/len(compare_data)
    print('accuracy on this weeks games')
    print(right_count/len(compare_data))
    
    total_squared_error = 0.0
    for tpl in compare_data:
        total_squared_error = total_squared_error + (tpl[0]-tpl[1])**2
    RMSE = (total_squared_error / float(len(compare_data)))**(0.5)
    print('RMSE on this weeks games:')
    print(RMSE)
    
    return fit, accuracy, RMSE, right_count, total_squared_error
     

In [33]:
#Now the code for running the week by week testing.
base_df = train_df
new_indices = []
# this is the hash for the first date
last_date_hash = 2018

fit_list = []
accuracy_list = []
RMSE_list = []
total_squared_error = 0
total_right_count = 0

for index, row in test_df.iterrows():
    
    year = row['year']
    month = row['month']
    day = row['day']
    date_hash = year+month+day 
    
    if date_hash != last_date_hash:
        last_date_hash = date_hash
        test_week = test_df.iloc[new_indices]
        fit, accuracy, RMSE, correct_calls, squared_error = train_test_model(base_df,test_week)
        
        fit_list.append(fit)
        accuracy_list.append(accuracy)
        RMSE_list.append(RMSE)
        
        total_squared_error = total_squared_error + squared_error
        total_right_count = total_right_count + correct_calls
        
        base_df = pd.concat([base_df,test_week],ignore_index=True)
        new_indices = [index]
        
    else:
        new_indices.append(index)

R^2 on the training data:
0.514115428151
accuracy on this weeks games
1.0
RMSE on this weeks games:
24.6130049653
R^2 on the training data:
0.515062529455
accuracy on this weeks games
0.75
RMSE on this weeks games:
8.50571284957
R^2 on the training data:
0.515637252787
accuracy on this weeks games
1.0
RMSE on this weeks games:
1.79624117389
R^2 on the training data:
0.515707166397
accuracy on this weeks games
1.0
RMSE on this weeks games:
7.80308140843
R^2 on the training data:
0.516755093582
accuracy on this weeks games
1.0
RMSE on this weeks games:
8.30907342967
R^2 on the training data:
0.530413242372
accuracy on this weeks games
0.6
RMSE on this weeks games:
24.0421530547
R^2 on the training data:
0.524662107499
accuracy on this weeks games
0.7307692307692307
RMSE on this weeks games:
18.2597176436
R^2 on the training data:
0.537844378003
accuracy on this weeks games
0.0
RMSE on this weeks games:
13.2061739345
R^2 on the training data:
0.537113491933
accuracy on this weeks games
0.

In [34]:
# get the number of games it called correctly in 2016
total_accuracy = total_right_count/test_df.shape[0]
total_accuracy

0.711038961038961

In [35]:
# get the Root Mean Squared Error
overall_RMSE = (total_squared_error/test_df.shape[0])**(0.5)
overall_RMSE

17.42496647588068